#### RSA

In [ ]:
openssl genpkey -algorithm RSA -out private_key.pem -pkeyopt rsa_keygen_bits:3072

In [ ]:
openssl rsa -pubout -in private_key.pem -out public_key.pem

In [ ]:
chmod 600 private_key.pem

In [ ]:
chmod 644 public_key.pem

#### Создание пары ключей RSA через Python

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
private_key = rsa.generate_private_key(public_exponent=65537,key_size=3072,backend=default_backend(), )
public_key = private_key.public_key()

#### Сохранение пары ключей RSA на Python

In [ ]:
private_bytes = private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption(), )
with open('private_key.pem', 'xb') as private_file:
    private_file.write(private_bytes)
public_bytes = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo, )
with open('public_key.pem', 'xb') as public_file:
    public_file.write(public_bytes)

#### Чтение пары ключей RSA на Python

In [ ]:
with open('private_key.pem', 'rb') as private_file:
    loaded_private_key = serialization.load_pem_private_key(
    private_file.read(),
    password=None,
    backend=default_backend()
    )
with open('public_key.pem', 'rb') as public_file:
    loaded_public_key = serialization.load_pem_public_key(
    public_file.read(),
    backend=default_backend()
    )

#### Шифрование парой ключей RSA на Python

In [1]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
padding_config = padding.OAEP(
    mgf=padding.MGF1(algorithm=hashes.SHA256()),
    algorithm=hashes.SHA256(),
    label=None, )
plaintext = b'message from Alice to Bob'
ciphertext = loaded_public_key.encrypt(
    plaintext=plaintext,
    padding=padding_config, )
decrypted_by_private_key = loaded_private_key.decrypt(
    ciphertext=ciphertext,
    padding=padding_config)
assert decrypted_by_private_key == plaintext

NameError: name 'loaded_public_key' is not defined

#### Подписание данных криптосистемой RSA на Python

In [2]:
import json
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes

message = b'from Bob to Alice'
padding_config = padding.PSS(
    mgf=padding.MGF1(hashes.SHA256()),
    salt_length=padding.PSS.MAX_LENGTH)
private_key = load_rsa_private_key()
signature = private_key.sign(
    message,
    padding_config,
    hashes.SHA256())
signed_msg = {
 'message': list(message),
 'signature': list(signature),
}
outbound_msg_to_alice = json.dumps(signed_msg)

NameError: name 'load_rsa_private_key' is not defined

In [ ]:
#### Проверка подписи, созданной криптосистемой RSA, на Python

In [3]:
import json
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.exceptions import InvalidSignature

def receive(inbound_msg_from_bob):
    signed_msg = json.loads(inbound_msg_from_bob)
    message = bytes(signed_msg['message'])
    signature = bytes(signed_msg['signature'])
    padding_config = padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH)
    private_key = load_rsa_private_key()
    try:
        private_key.public_key().verify(
            signature,
            message,
            padding_config,
            hashes.SHA256())
        print('Сообщению можно доверять')
    except InvalidSignature:
        print('Сообщению нельзя доверять')

#### Подписание данных на базе эллиптических кривыхm